## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
dir_path = "/Users/ecem/Desktop/seperate-axis"

In [4]:
z = np.load(dir_path + "/z-10sec.npy", allow_pickle= True)

In [5]:
print(z.shape)


(2392, 2560)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [7]:
lbs.feature.zero_crossing_rate(np.array(z[7]))

array([[0.09667969, 0.14697266, 0.20605469, 0.20849609, 0.15917969,
        0.109375  ]])

In [8]:
sum(lbs.zero_crossings(np.array(z[7])))

526

In [9]:
z.shape

(2392, 2560)

In [10]:
def zero_crossing(array):
    zero_crossing_rate = np.ones(shape = (array.shape[0], 7))
    
    for i in range(array.shape[0]):
        zero_crossing_rate[i][0:6] = lbs.feature.zero_crossing_rate(np.array(array[i]))
        zero_crossing_rate[i][6] = sum(lbs.zero_crossings(np.array(array[i])))
    return zero_crossing_rate

        

In [11]:
zcr_z = zero_crossing(z)

In [12]:
print(zcr_z.shape)

(2392, 7)


In [13]:
zcr_z[0]

array([8.54492188e-02, 1.45507812e-01, 1.94335938e-01, 2.06542969e-01,
       1.62597656e-01, 1.03027344e-01, 5.10000000e+02])

In [15]:
columns_zcr = []
for i in range(zcr_z.shape[1]-1):
    columns_zcr.append("zcr_z{}".format(i))

    
columns_zcr.append("#ofcrossing_z")

print(columns_zcr)

['zcr_z0', 'zcr_z1', 'zcr_z2', 'zcr_z3', 'zcr_z4', 'zcr_z5', '#ofcrossing_z']


In [16]:
def dataframe_creator(array, df = pd.DataFrame(), l = 7, columns = None):
    values = []

    for i in range(array.shape[0]):
        llist = list(array[i][0:l-1].tolist())
        llist.append(array[i][l-1])
        
        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [17]:
df = dataframe_creator(zcr_z, columns = columns_zcr)

df.reset_index(drop = True, inplace = True)
df

,zcr_z0,zcr_z1,zcr_z2,zcr_z3,zcr_z4,zcr_z5,#ofcrossing_z
0,0.085449,0.145508,0.194336,0.206543,0.162598,0.103027,510.0
1,0.104004,0.159668,0.220215,0.236816,0.179688,0.124023,582.0
2,0.126465,0.180664,0.234863,0.230957,0.162109,0.107910,593.0
3,0.120605,0.169922,0.224609,0.216309,0.160156,0.110352,576.0
4,0.111328,0.155762,0.200684,0.192871,0.136719,0.091797,509.0
...,...,...,...,...,...,...,...
2387,0.109375,0.174805,0.230957,0.234863,0.182129,0.116211,598.0
2388,0.111328,0.163574,0.212402,0.212402,0.163086,0.110840,563.0
2389,0.113770,0.171875,0.222168,0.206543,0.154297,0.096191,550.0
2390,0.120605,0.174316,0.225586,0.221680,0.162598,0.109375,582.0


In [27]:
df.to_csv(dir_path + '/z-axis-zcrs.csv')

### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
import librosa.display

In [30]:
lbs.feature.chroma_stft(np.array(z[i]), sr= 256, hop_length = 512)[1].mean()

0.8474578191769702

In [31]:
def chromagram_average(array):
    pitch = np.ones(shape = (array.shape[0], 12)) 
    
    for i in range(array.shape[0]):
        for j in range(12):
            pitch[i][j] = lbs.feature.chroma_stft(np.array(array[i]), sr= 256, hop_length = 512)[j].mean()
    return pitch

In [32]:
chro_z = chromagram_average(z)

In [33]:
print(chro_z.shape)

(2392, 12)


In [34]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [35]:
col_chro = []
for i in range(chro_z.shape[1]):
    col_chro.append(lst[i]+ "_z")
    
print(col_chro)
print(len(col_chro))

['C_z', 'C♯_z', 'D_z', 'D♯_z', 'E_z', 'F_z', 'F♯_z', 'G_z', 'G♯_z', 'A_z', 'A♯_z', 'B_z']
12


In [36]:
ch_df = dataframe_creator(chro_z,
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C_z,C♯_z,D_z,D♯_z,E_z,F_z,F♯_z,G_z,G♯_z,A_z,A♯_z,B_z
0,0.7502,0.742305,0.63363,0.622867,0.56488,0.544785,0.482642,0.468954,0.532528,0.74158,0.855271,1.0
1,0.853189,0.855843,0.844548,0.867438,0.752847,0.601679,0.559608,0.5189,0.58309,0.768403,0.938481,0.981522
2,0.849784,0.751048,0.710831,0.591588,0.532734,0.5244,0.476397,0.448132,0.493588,0.681753,0.913051,1.0
3,0.747486,0.688325,0.676742,0.587279,0.543566,0.535861,0.476171,0.485026,0.560095,0.714123,0.848634,1.0
4,0.801979,0.879693,0.717716,0.616657,0.530958,0.405941,0.454041,0.454407,0.528925,0.765874,0.837774,0.999276
...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.982692,0.760464,0.760573,0.732119,0.651387,0.647699,0.509018,0.532665,0.677828,0.745018,0.735538,0.984908
2388,0.888338,0.786794,0.606148,0.544596,0.525811,0.487809,0.477325,0.537382,0.569543,0.601511,0.753724,0.966602
2389,0.963077,0.873883,0.670385,0.558793,0.432298,0.490846,0.470305,0.471849,0.482405,0.600722,0.668828,0.863238
2390,0.995919,0.799401,0.613326,0.58971,0.574288,0.496267,0.549003,0.620214,0.602249,0.586552,0.617611,0.770166


In [37]:
ch_df.to_csv(dir_path + "/z-axis-chromogram.csv")

## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [38]:
def tempo_average(array): # 384, 11
    means= []
    for i in range(array.shape[0]): #384
        means.append(array[i][0:11].mean())
    return np.array(means) # 384
        

In [39]:
def tempogram(array = 0, hop_lenght = 256):
    hop_length = 256
    sr =256
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sr, 
                                            hop_length=hop_length)

    times = librosa.times_like(oenv, sr=sr, hop_length=hop_length)

    tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr= sr,
                                              hop_length=hop_length)
    # Estimate the global tempo for display purposes
    tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sr,
                                   hop_length=hop_length)[0]
    return [tempogram, tempo]

In [40]:
tempogram(z[0])[0].shape

(384, 11)

In [41]:
tempogram(z[0])[0][12]

array([0.41895497, 0.42113974, 0.42333447, 0.42553937, 0.42775463,
       0.42998046, 0.43221704, 0.43446459, 0.4367233 , 0.43899338,
       0.44127505])

In [42]:
z[0].shape

(2560,)

In [43]:
def temp_values(array):
    final = np.ones(shape = (array.shape[0], 384))
    for i in range(array.shape[0]): #143
        
        _x = tempogram(array[i])[0] #1,2560

        final[i] = tempo_average(_x) #384
        
        
    return final

In [44]:
temp_z = temp_values(z)


In [45]:
temp_z.shape

(2392, 384)

In [46]:
columns_t = []
for i in range(temp_z.shape[1]):
    columns_t.append("tempogram_z{}".format(i))
print(columns_t)

['tempogram_z0', 'tempogram_z1', 'tempogram_z2', 'tempogram_z3', 'tempogram_z4', 'tempogram_z5', 'tempogram_z6', 'tempogram_z7', 'tempogram_z8', 'tempogram_z9', 'tempogram_z10', 'tempogram_z11', 'tempogram_z12', 'tempogram_z13', 'tempogram_z14', 'tempogram_z15', 'tempogram_z16', 'tempogram_z17', 'tempogram_z18', 'tempogram_z19', 'tempogram_z20', 'tempogram_z21', 'tempogram_z22', 'tempogram_z23', 'tempogram_z24', 'tempogram_z25', 'tempogram_z26', 'tempogram_z27', 'tempogram_z28', 'tempogram_z29', 'tempogram_z30', 'tempogram_z31', 'tempogram_z32', 'tempogram_z33', 'tempogram_z34', 'tempogram_z35', 'tempogram_z36', 'tempogram_z37', 'tempogram_z38', 'tempogram_z39', 'tempogram_z40', 'tempogram_z41', 'tempogram_z42', 'tempogram_z43', 'tempogram_z44', 'tempogram_z45', 'tempogram_z46', 'tempogram_z47', 'tempogram_z48', 'tempogram_z49', 'tempogram_z50', 'tempogram_z51', 'tempogram_z52', 'tempogram_z53', 'tempogram_z54', 'tempogram_z55', 'tempogram_z56', 'tempogram_z57', 'tempogram_z58', 'tempo

In [47]:
len(columns_t)

384

In [48]:
df_t = dataframe_creator(temp_z,
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 384)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_z0,tempogram_z1,tempogram_z2,tempogram_z3,tempogram_z4,tempogram_z5,tempogram_z6,tempogram_z7,tempogram_z8,tempogram_z9,...,tempogram_z374,tempogram_z375,tempogram_z376,tempogram_z377,tempogram_z378,tempogram_z379,tempogram_z380,tempogram_z381,tempogram_z382,tempogram_z383
0,1.0,0.655776,0.561135,0.5306,0.478565,0.465403,0.460404,0.455384,0.450346,0.44529,...,-0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0
1,1.0,0.698621,0.558041,0.500824,0.502186,0.5113,0.505712,0.500104,0.494477,0.488832,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,1.0,0.72352,0.628894,0.625713,0.628625,0.62615,0.619122,0.612071,0.605001,0.597912,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,1.0,0.683446,0.558907,0.529697,0.512323,0.508083,0.502545,0.496987,0.491409,0.485814,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,1.0,0.72168,0.620978,0.598515,0.594308,0.581373,0.574944,0.568493,0.562021,0.555532,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,1.0,0.739676,0.664985,0.662011,0.647993,0.630146,0.623118,0.616066,0.608993,0.601902,...,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2388,1.0,0.667492,0.532212,0.495928,0.466804,0.436559,0.431911,0.427244,0.422558,0.417855,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2389,1.0,0.679825,0.573402,0.553152,0.55128,0.547799,0.541751,0.535682,0.529593,0.523487,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2390,1.0,0.648589,0.529137,0.492979,0.483272,0.468527,0.463463,0.458379,0.453276,0.448157,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [49]:
df_t.to_csv(dir_path + '/z-axis-tempogram.csv')

In [50]:
df_t.isin([0]).sum(axis=0)

tempogram_z0      0
tempogram_z1      0
tempogram_z2      0
tempogram_z3      0
tempogram_z4      0
                 ..
tempogram_z379    0
tempogram_z380    0
tempogram_z381    0
tempogram_z382    0
tempogram_z383    0
Length: 384, dtype: int64

In [51]:
df_t['tempogram_z382'][10]

6.384126339617951e-17

In [52]:
final_df = pd.concat([df, ch_df], axis =1)
final_df

,zcr_z0,zcr_z1,zcr_z2,zcr_z3,zcr_z4,zcr_z5,#ofcrossing_z,C_z,C♯_z,D_z,D♯_z,E_z,F_z,F♯_z,G_z,G♯_z,A_z,A♯_z,B_z
0,0.085449,0.145508,0.194336,0.206543,0.162598,0.103027,510.0,0.7502,0.742305,0.63363,0.622867,0.56488,0.544785,0.482642,0.468954,0.532528,0.74158,0.855271,1.0
1,0.104004,0.159668,0.220215,0.236816,0.179688,0.124023,582.0,0.853189,0.855843,0.844548,0.867438,0.752847,0.601679,0.559608,0.5189,0.58309,0.768403,0.938481,0.981522
2,0.126465,0.180664,0.234863,0.230957,0.162109,0.107910,593.0,0.849784,0.751048,0.710831,0.591588,0.532734,0.5244,0.476397,0.448132,0.493588,0.681753,0.913051,1.0
3,0.120605,0.169922,0.224609,0.216309,0.160156,0.110352,576.0,0.747486,0.688325,0.676742,0.587279,0.543566,0.535861,0.476171,0.485026,0.560095,0.714123,0.848634,1.0
4,0.111328,0.155762,0.200684,0.192871,0.136719,0.091797,509.0,0.801979,0.879693,0.717716,0.616657,0.530958,0.405941,0.454041,0.454407,0.528925,0.765874,0.837774,0.999276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.109375,0.174805,0.230957,0.234863,0.182129,0.116211,598.0,0.982692,0.760464,0.760573,0.732119,0.651387,0.647699,0.509018,0.532665,0.677828,0.745018,0.735538,0.984908
2388,0.111328,0.163574,0.212402,0.212402,0.163086,0.110840,563.0,0.888338,0.786794,0.606148,0.544596,0.525811,0.487809,0.477325,0.537382,0.569543,0.601511,0.753724,0.966602
2389,0.113770,0.171875,0.222168,0.206543,0.154297,0.096191,550.0,0.963077,0.873883,0.670385,0.558793,0.432298,0.490846,0.470305,0.471849,0.482405,0.600722,0.668828,0.863238
2390,0.120605,0.174316,0.225586,0.221680,0.162598,0.109375,582.0,0.995919,0.799401,0.613326,0.58971,0.574288,0.496267,0.549003,0.620214,0.602249,0.586552,0.617611,0.770166


In [53]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcr_z0,zcr_z1,zcr_z2,zcr_z3,zcr_z4,zcr_z5,#ofcrossing_z,C_z,C♯_z,D_z,...,tempogram_z374,tempogram_z375,tempogram_z376,tempogram_z377,tempogram_z378,tempogram_z379,tempogram_z380,tempogram_z381,tempogram_z382,tempogram_z383
0,0.085449,0.145508,0.194336,0.206543,0.162598,0.103027,510.0,0.7502,0.742305,0.63363,...,-0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0
1,0.104004,0.159668,0.220215,0.236816,0.179688,0.124023,582.0,0.853189,0.855843,0.844548,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,0.126465,0.180664,0.234863,0.230957,0.162109,0.107910,593.0,0.849784,0.751048,0.710831,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.120605,0.169922,0.224609,0.216309,0.160156,0.110352,576.0,0.747486,0.688325,0.676742,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.111328,0.155762,0.200684,0.192871,0.136719,0.091797,509.0,0.801979,0.879693,0.717716,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,0.109375,0.174805,0.230957,0.234863,0.182129,0.116211,598.0,0.982692,0.760464,0.760573,...,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2388,0.111328,0.163574,0.212402,0.212402,0.163086,0.110840,563.0,0.888338,0.786794,0.606148,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2389,0.113770,0.171875,0.222168,0.206543,0.154297,0.096191,550.0,0.963077,0.873883,0.670385,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2390,0.120605,0.174316,0.225586,0.221680,0.162598,0.109375,582.0,0.995919,0.799401,0.613326,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [54]:
final_df.to_csv(dir_path + '/z-axis-final-signal_processing.csv')